In [9]:
import numpy as np
from numpy.linalg import norm, solve, multi_dot
import matplotlib.pyplot as plt
from scipy.optimize import minimize_scalar

In [43]:
# define the function
g = lambda x,y: (x-1)**2 + (2*y-1)**2
# define derivatives of f to make the gradient
Dg = lambda x,y: np.array([2*(x-1), 4*(2*y-1)])

In [44]:
x = (4,1)       # initial point
path = [x]
print(f'Initial x={x}')
dx = np.array([np.inf,np.inf]) # initial large gradient so while loop runs
tol = 1e-8          # stop when gradient is smaller than this amount
max_steps = 1000      # Maximum number of steps to run the iteration       
i=0                   # iteration countz
while np.linalg.norm(dx)>tol and i<max_steps:
    dx = Dg(x[0],x[1])
    
    # choose a2=1/5
    a = 1.425

    # new value of x
    xnew = x - a*dx
    path.append(xnew)
    
    # update old value
    x = xnew
    # update iteration count
    i += 1

path=np.array(path)
print(f'After {i} iterations, approximate minimum is {g(x[0],x[1])} at {x}')
xnew = x - a*dx
con=(np.linalg.norm(xnew-(1,0.5)))/(np.linalg.norm(x-(1,0.5))) 
con

Initial x=(4, 1)
After 59 iterations, approximate minimum is 1.0978887356815322e-17 at [1.  0.5]


0.5815603231354938

In [19]:
#define Rosenbrock, its gradient and Hessian
def RBObjFunc(x):
    return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2
def RBGradObjFunc(x):
    return np.array([
        -400 * x[0] * (x[1] - x[0] ** 2) - 2 * (1 - x[0]),
        200 * (x[1] - x[0]**2)      
    ])
def RBHessianFunc(x):
    return np.matrix([
        [-400*(x[1] - 3*x[0]**2) + 2, -400 * x[0]], [-400 * x[0], 
200]
    ])

In [20]:
def WolfeI(alpha,f,x,dx,p,c1=0.1):
    '''Return True/False if Wolfe condition I is satisfied for the given alpha'''
    LHS = f(x[0]+alpha*p[0], x[1]+alpha*p[1])
    RHS = f(x[0],x[1])+c1*alpha*np.dot(dx(x[0],x[1]),p)
    return LHS <= RHS

In [21]:
def newton_method(objFunc, gradObjFunc, hessianFunc, x0, 
tol, maxIter):                      
    path      = [x0]
    k         = 0
    xk        = x0    
    pk        = -np.linalg.solve(hessianFunc(xk[0],xk[1]), gradObjFunc(xk[0],xk[1]))        
    while norm(gradObjFunc(xk[0],xk[1])) > tol and k <= maxIter:  
        # backtracking
       
        alpha = 1
        while not WolfeI(alpha,objFunc,xk,gradObjFunc,pk) and alpha>1e-5:  # lower limit to prevent small steps, similar to Wolfe II
            alpha *= rho   
        

        xk  = xk + alpha*pk
        
        pk  = -np.linalg.solve(hessianFunc(xk[0],xk[1])  , gradObjFunc(xk[0],xk[1]))
        k   = k + 1
        print("At {iter} iterations, the step length is {aa}.".format(iter=k, aa=alpha))
        path.append(xk)
        
    path = np.array(path) # convert to array
        
    if norm(pk) <= tol:
        print("Found the minimizer at {x} with {iter} iterations successfully, gradient's norm is {nrm}.".format(x=xk,iter=k,nrm=norm(pk)))
    else:
        print("Unable to locate minimizer within maximum iterations, last position is at {x}, gradient's norm is {nrm}".format(x=xk,nrm=norm(pk)))
        
    return xk, k, path

In [22]:
#define Rosenbrock, its gradient and Hessian
RBObjFunc= lambda x,y: 100 * (y - x**2)**2 + (1 - x)**2
RBGradObjFunc=lambda x,y: np.array([
        -400 * x * (y - x ** 2) - 2 * (1 - x),
        200 * (y - x**2)      
    ])
RBHessianFunc=lambda x,y: np.matrix([
        [-400*(y - 3*x**2) + 2, -400 * x], [-400 * x, 200]])

In [25]:
#initialize, input parameters
x0      = np.array([1.2, 1.2]) 
tol     = 1e-8 
maxIter = 1e6
rho=0.75

In [26]:
##Run
x_2, iter_2, path_2 = newton_method(RBObjFunc, RBGradObjFunc, RBHessianFunc, x0, tol, maxIter)

At 1 iterations, the step length is 1.
At 2 iterations, the step length is 0.5625.
At 3 iterations, the step length is 1.
At 4 iterations, the step length is 1.
At 5 iterations, the step length is 1.
At 6 iterations, the step length is 1.
At 7 iterations, the step length is 1.
At 8 iterations, the step length is 1.
Found the minimizer at [1. 1.] with 8 iterations successfully, gradient's norm is 4.965068306494916e-16.


In [28]:
def exact_line_search_quasi_newton(update_method, x0, H0,tol):
    k = 0
    xcoords = [x0[0]]
    ycoords = [x0[1]]
    path = [[x0[0],x0[1]]]
    x_k = x0 
    H_k = H0 
    g_k = grad_objective_func(x_k)
    while norm(g_k) > tol:
        p_k = -np.matmul(H_k, g_k)                # search direction
   
        def subproblem1D(alpha):                  # exact line search
            return objective_func(x_k + alpha * p_k)
        
        res = minimize_scalar(subproblem1D)
        alpha_k = res.x      
        s_k     = alpha_k * p_k                   # s_k = x_{k+1} - x_k 
        g_k1    = grad_objective_func(x_k + s_k)  # gk1 is g_{k+1}
        y_k     = g_k1 - g_k                      # y_k = g_{k+1} - g_(k)
        
        k = k + 1                                 # increment
        H_k = update_method(H_k, s_k, y_k).A      # .A transforms       
#matrix  to ndarray 
        x_k = x_k + s_k 
        g_k = g_k1
        path.append([x_k[0],x_k[1]])
        xcoords.append(x_k[0])
        ycoords.append(x_k[1])
        
    return x_k, k, norm(g_k), xcoords, ycoords ,path

In [29]:
def DFP(H, s_k, y_k):
    rho = 1.0 / np.dot(s_k, y_k)
    vec = H*np.matrix(y_k).T
    denom2 = np.matrix(y_k)*vec 
    H = H + rho*(np.matrix(s_k).T * np.matrix(s_k)) - np.matrix(vec)*np.matrix(vec).T/denom2 # np.outer(vec, vec) = vec * vec^T #H^T=H
    return H

In [30]:
def BFGS(H, s_k, y_k): 
    rho = 1.0 / np.dot(s_k, y_k)
    I = np.eye(len(s_k))
    A = I - rho * (np.matrix(s_k).T * np.matrix(y_k) )
    B = np.matrix(s_k).T * np.matrix(s_k) * rho
    H = A * H * A.T + B 
    return H

In [31]:
def SR1(H, s_k, y_k):
    z = s_k - np.dot(H, y_k)     # the numerator is a matrix
    numer = np.matrix(z).T * np.matrix(z)
    denom = np.dot(z, y_k)
    # skip when the denominator is too small
    if np.abs(denom) < 10**(-8) * norm(z) * norm(y_k):
        return H
    else:
        return H + numer/denom 

In [32]:
def objective_func(x):
    
     return 100 * (x[1] - x[0]**2)**2 + (1 - x[0])**2
def grad_objective_func(x):
    
         return np.array([
        -400 * x[0] * (x[1] - x[0] ** 2) - 2 * (1 - x[0]),
        200 * (x[1] - x[0]**2)      
    ])


In [45]:
x0 = np.array([1.2, 1.2])
H0 = np.eye(2)
xB, iter_numberB, errB, xcoordsB, ycoordsB,pathB  = exact_line_search_quasi_newton(BFGS, x0, H0,tol = 1e-8)
xS, iter_numberS, errS, xcoordsS, ycoordsS,pathS = exact_line_search_quasi_newton(SR1, x0, H0,tol = 1e-8)
xD, iter_numberD, errD, xcoordsD, ycoordsD,pathD = exact_line_search_quasi_newton(DFP, x0, H0,tol = 1e-8)

In [46]:
def approx_convergence_rate(xcoords,ycoords, minimizer):
    '''Given a path defined by an iteration and a known minimizer, approximates convergence rate'''
    p=np.log(np.linalg.norm(np.array(xcoords[-1],ycoords[-1])-minimizer
                           )/(np.linalg.norm(np.array(xcoords[-2],ycoords[-2])-minimizer))
            )/(np.log(np.linalg.norm(np.array(xcoords[-2],ycoords[-2])-minimizer
    )/(np.linalg.norm(np.array(xcoords[-3],ycoords[-3])-minimizer))))
    return p

In [47]:
def approx_convergence_rate1(xcoords,ycoords):
    '''Given a path defined by an iteration and a known minimizer, approximates convergence rate'''
    p=np.log(np.linalg.norm(np.array(xcoords[-1],ycoords[-1])-np.array(xcoords[-2],ycoords[-2])
                           )/(np.linalg.norm(np.array(xcoords[-2],ycoords[-2])-np.array(xcoords[-3],ycoords[-3])))
            )/(np.log(np.linalg.norm(np.array(xcoords[-2],ycoords[-2])-np.array(xcoords[-3],ycoords[-3])
                                     )/(np.linalg.norm(np.array(xcoords[-3],ycoords[-3])
                                                       -np.array(xcoords[-4],ycoords[-4])))))
    return p

In [48]:
pB=approx_convergence_rate1(xcoordsB,ycoordsB,(1,1))
pB

TypeError: approx_convergence_rate1() takes 2 positional arguments but 3 were given

In [49]:
pS=approx_convergence_rate1(xcoordsS,ycoordsS,(1,1))
pB

TypeError: approx_convergence_rate1() takes 2 positional arguments but 3 were given

In [40]:
pS=approx_convergence_rate1(xcoordsS,ycoordsS,(1,1))
pS

TypeError: approx_convergence_rate1() takes 2 positional arguments but 3 were given

In [ ]:
def approx_convergence_rate(path, minimizer, numToAvg=3 showPlot=True):
    '''Given a path defined by an iteration and a known minimizer, approximates convergence rate'''
    err = np.linalg.norm(path-np.array(minimizer),axis=1) # ||x_k-x*||=e_k
    
    # if converged in very few steps, return infinite order
    if len(err)<=3:
        return np.inf
    
    pp = np.zeros(len(err)-3)
    for i in range(len(err)-3):
        pp[i] = np.log(err[i+2]/err[i+1])/np.log(err[i+1]/err[i])
    
    if numToAvg>len(pp):
        # if not enough iterations to average, just average all
        p=np.mean(pp)
    else:
        # return mean of last few iterations
        p=np.mean(pp[-numToAvg:])
        
    # plot
    if showPlot:
        plt.plot(pp)
        plt.plot(pp*0+p)
        plt.xlabel('k')
        plt.ylabel('p')
        plt.title(f'p={p}')
        plt.show()
        
    return p